#### 哨兵和 cluster
17 ~ 28  
为什么 redis 集群有 16384 个槽位 https://blog.csdn.net/kingmax54212008/article/details/104090790



### cache aside 缓存一致性
#### 1. 最简单的做法   
读操作: 先读 redis
* 读到: 直接返回
* 未读到: 去读 Mysql, mysql 返回的数据添加到 redis

更新操作: 先删 redis, 再更新 mysql

**[问1]**: 问什么要先删除 redis 而不是直接更新 redis?        
因为 redis 中的缓存数据有时需要经过复杂运算后得出, 先删除后在读到时再插入 redis 的做法, 避免每次更新都进行复杂的缓存计算过程, 同时这也符合数据的懒加载原则.

**[问2]**: 先更新 mysql, 再删除 redis 可以吗?    
不行, 因为删除 redis 如果放在第二步做, 一旦发生删除失败, 导致缓存中还是旧数据, 与 mysql 不一致  
如果删除 redis 再第一步做, 删除失败报错会导致进行不到写 mysql, 虽然操作失败, 但数据在 mysql 和 redis 是一致的

#### 2. 高并发场景下的缓存双写不一致 (34)
用内存队列讲异步操作串行化





### 缓存雪崩的解决方案  (35)
原因: redis 集群崩溃, 访问 redis 缓存会等待超时报错   
后果:   
(1) 原先访问 redis 只需要5ms, 超时导致服务 hang 住等待超时达到 5s,占用 tomcat 大量线程资源  
(2) redis 超时报错后, 所有访问直接访问 mysql, 导致 mysql 又会挂掉, 后面访问 mysql 又会超时, 继续占用 tomcat 大量现成资源  
(3) nginx 一般也有缓存, nginx 缓存超时后访问 redis 失效, 导致 nginx 缓存更新失败, 用户访问网站显示超时, 页面无法访问.  

事前事中事后三个层次的缓存雪崩解决方案 (36)   
* `事前`: 从 redis 的部署上解决高可用   
	redis cluster    
* `事中`: redis cluster 已经崩溃了, 大量访问已经无法访问 redis.    
	(1) 应该对 redis 的访问操作, 加上 spring cloud sentinel 的`降级策略`.    
      (比如降级为访问本地缓存 ehcache, 商品预览服务降级的访问 ehcache 的话, 仍然能看到热点商品)       
	(2) ehcache 如果也找不到数据,就要发出系统调用或 mysql 查询, 应该对着两个访问操作做`限流`, 限制访问服务/mysql的 qps, 避免其它服务的 mysql 宕机    
* `事后`:   
	(1) 有备份数据的话, 使用 AOF 或 RDB 恢复 redis cluster 的备份数据    
	(2) 没有备份数据的话, 可以使用"缓存快速预热", 然后再把 redis 彻底启动起来      





#### 缓存穿透 (38)


#### 分布式锁 (40,72,75)





#### 秒杀 (50, 77 (1,2,3,4), 83, 84)



#### 热点 (89,92,93,94,95)


#### 多级缓存系统 (31,32)


